In [1]:
import numpy as np
import pandas as pd
import json
import codecs
import datetime
import os
import sys
sys.path.append('../')
import matplotlib as mpl
import matplotlib.pyplot as plt
from vector_creator.raw_to_df.rawdata_to_df import *
from vector_creator.score_vectors.score_vectors_assembly import *
from vector_creator.stats_models.entropy import *
from vector_creator.preprocess.est_by_df_column import *
from vector_creator.stats_models.auto_regression import *
from vector_creator.preprocess.install_apps_features import *
from vector_creator.stats_models.estimators import *
from vector_creator.score_vectors.vector_indexer import vector_desc_call_logs
from vector_creator.score_vectors.vector_descriptor import *
path = '/Users/smuel/AndroidStudioProjects/LuckyMeIntgratedSDK/metadata_init_phase/'

In [2]:
rdata = list_of_json_files(path)
rdata[0][0]

'05fa68e5-0ba3-4783-9c07-42740c4914c6'

In [3]:
raw = group_metadata(path)
uid  = '05fa68e5-0ba3-4783-9c07-42740c4914c6'
file = '05fa68e5-0ba3-4783-9c07-42740c4914c6_11-06-2021_09-53-01.json'
raw_data = json.load(codecs.open(path + file, 'r', 'utf-8-sig'))

In [4]:
raw_data.keys()

dict_keys(['SamplingConfigurations', 'CallLogs', 'ImgMetaData', 'InstallApps', 'LocationInfo', 'BatteryInfo', 'WifiInfo', 'ScreenInfo'])

In [5]:
loc, df_r = create_df_from_init_metadata(uid, raw_data)

In [6]:
df_r.keys()

dict_keys(['05fa68e5-0ba3-4783-9c07-42740c4914c6_SamplingConfigurations', '05fa68e5-0ba3-4783-9c07-42740c4914c6_CallLogs', '05fa68e5-0ba3-4783-9c07-42740c4914c6_ImgMetaData', '05fa68e5-0ba3-4783-9c07-42740c4914c6_InstallApps', '05fa68e5-0ba3-4783-9c07-42740c4914c6_LocationInfo', '05fa68e5-0ba3-4783-9c07-42740c4914c6_BatteryInfo', '05fa68e5-0ba3-4783-9c07-42740c4914c6_WifiInfo', '05fa68e5-0ba3-4783-9c07-42740c4914c6_ScreenInfo'])

In [7]:
df=df_r['05fa68e5-0ba3-4783-9c07-42740c4914c6_ImgMetaData']

In [8]:
df.head(5)

,IMAGE_TYPE,IMAGE_DATE_TIME
0,image/jpeg,2019-10-16 08:48:55
1,image/jpeg,2019-10-16 08:49:06
2,image/jpeg,2019-10-16 08:49:07
3,image/jpeg,2019-10-20 15:42:48
4,image/jpeg,2019-10-22 14:09:27


In [9]:
loc_tuple = (loc[0]['Latitude'], loc[0]['Longitude'])

In [10]:
x = score_vector_constructor(path, 'photo-gallery')

(21, 47)


In [12]:
x.iloc[:, 30:43]

description,ratio of night hours and daily mean number of photos,ratio of day hours and daily mean number of photos,ratio of night hours and day hours mean number of photos,ratio of weekend and week daily mean number of photos,ratio of week work days and week daily mean number of photos,ratio of weekend and week work days daily mean number of photos,ratio of night hours and daily mean photos in burst,ratio of night hours and daily mean bursts a week,ratio of weekend and full week mean photos in burst,ratio of weekend and full week bursts a week,ar of num of photos in day time lag-1,entropy on number of photos taken in day,mean max ratio of category of installed apps
05fa68e5-0ba3-4783-9c07-42740c4914c6,1.086989,0.789017,1.377650,1.412923,0.850437,1.661408,1.067299,0.558287,1.143289,0.360157,0.925484,6.535033,0.325397
12fa7561-2e77-4334-b9e1-6858ed27570f,0.191247,1.462908,0.130730,2.531653,0.212293,11.925287,0.260582,0.073910,2.181962,0.091065,5.376335,1.900352,0.345455
1362474c-0fb5-4932-a33d-5a30f23ac7e3,0.948417,0.753555,1.258591,1.338639,0.852766,1.569762,0.958638,0.590706,0.900341,0.992283,2.521008,5.911040,0.500000
148b0326-09e2-404f-9de6-aff6d44f2a38,0.880519,0.704600,1.249672,1.086207,0.959016,1.132626,0.892010,1.146786,0.910000,0.866667,0.681415,5.570877,0.279412
17ac0e20-b292-492c-9adc-faf60a068926,1.567834,0.321362,4.878725,1.985245,0.400286,4.959569,1.192369,0.036428,1.259321,0.084507,1.556856,3.365487,0.600000
199ea69c-5e20-4b53-9bfb-0534aaeb68d6,0.712657,1.133935,0.628481,0.911736,1.053493,0.865441,0.843879,0.579520,0.904728,0.519570,5.201939,4.374915,0.267857
1abd59a3-553f-43db-a329-90c28c72afdd,0.800241,1.037327,0.771444,1.193816,0.926266,1.288848,0.902978,0.248557,0.968735,0.226738,0.419881,6.308478,0.340000
20380cd2-055e-4f3a-a339-90a1c7df4e1a,0.669969,0.685745,0.976994,1.153651,0.951691,1.212212,0.926636,2.556678,1.073796,1.009259,20.128133,7.213645,0.151630
236b91d2-cefb-405a-9de5-a3673ef1c736,0.965670,0.427267,2.260109,0.581694,1.171613,0.496490,0.881250,4.250438,0.677958,0.665794,11.254550,4.955326,0.297101
25174909-3427-42d2-9e14-65f7c95f240b,0.529720,0.917651,0.577257,1.566858,0.805490,1.945224,0.977022,0.977843,0.897083,1.100073,6.749863,5.629823,0.000000


In [ ]:
#read the tags from json file for user register non register and create df with tag column
def create_tag_df(json_file_path, column_name): # 'entries.json'
    tag_data = json.load(codecs.open(json_file_path, 'r', 'utf-8-sig'))
    df_tag = pd.DataFrame(tag_data)
    df_tag = df_tag.rename(columns={column_name: ''}) # 'app_userid'
    dftag = df_tag.set_index('')
    return dftag

In [13]:
def norm_df(old_df):
    inxs = old_df.index.values
    zs = zscore(old_df.values, axis=0, ddof=1)
    return pd.DataFrame(zs, columns=old_df.columns, index=inxs)
n_df = norm_df(x)

In [14]:
def norm_df_2(df):
    inxs = df.index.values
    std_scale = preprocessing.StandardScaler().fit(df)
    df_std = std_scale.transform(df)
    return pd.DataFrame(df_std, columns=df.columns, index=inxs)
n2_df = norm_df_2(x)

In [27]:
def minmax_norm(df):
    inxs = df.index.values
    minmax_scale = preprocessing.MinMaxScaler().fit(df)
    df_minmax = minmax_scale.transform(df)
    return pd.DataFrame(df_minmax, columns=df.columns, index=inxs)
mm_df = minmax_norm(x)

In [35]:
mm_df.max(axis=0)

description
median number photos taken in week - location      1.0
mad number photos taken in week - scale            1.0
sample weeks ratio                                 1.0
mean number of photos in burst                     1.0
mean number of bursts a week                       1.0
median night time number photos                    1.0
mad night time number photos                       1.0
sampled week nights ratio                          1.0
entropy on number of photos taken in night time    1.0
ar of num of photos in night time lag-1            1.0
mean number of photos in burst at night            1.0
mean number of bursts a week nights                1.0
median day time number photos                      1.0
mad day time number photos                         1.0
sampled week days ratio                            1.0
entropy on number of photos taken in day time      1.0
ar of num of photos in day time lag-1              1.0
mean number of photos in burst at daytime          1.

In [26]:
n_df.values[:,0].mean()

2.1224851941363285e-17

In [18]:
dftag = create_tag_df('entries.json', 'app_userid')
dftag

,User_Register
,
005b6c49-c4ed-438b-9092-ad02ed4d672e,false
005d6e1d-7a19-422f-a71a-1555b6ca5724,false
007a0785-1321-4112-be89-e6549a4725a2,true
008e4a51-2ded-4a7f-a822-e1a4756d7ebd,false
00c86b27-f076-4560-bb3d-2fb0364b23a1,false
...,...
ff992f28-a57d-400d-87eb-5249fdc84337,true
ffc7b3ed-d66e-4075-ba35-aa1b96c88db7,false
ffccdd4d-03a1-41ba-bfcf-5f1428db631b,false


In [19]:
len(dftag)

2976

In [20]:
dfjoin = n_df.join(dftag)
dfjoin['User_Register'] = dfjoin['User_Register'].fillna('false')

In [22]:
len(dfjoin[dfjoin.User_Register == 'false'])

13

In [25]:
dfjoin.values[:,0].std(), dfjoin.values[:,0].mean()

(0.970142500145332, 2.7755575615628914e-17)

In [2]:
x = pd.read_csv('/Users/smuel/Developer/Python/csv_fles/photogallery_with_tag_for_xgboost.csv')

In [6]:
x = pd.read_csv('/Users/smuel/Developer/Python/csv_fles/photogallery_with_tag_for_xgboost.csv')


In [7]:
x.rename(columns={'Unnamed: 0': 'description'}, inplace=True)

In [11]:
x.head(5)

,median number photos taken in week - location,mad number photos taken in week - scale,sample weeks ratio,mean number of photos in burst,mean number of bursts a week,median night time number photos,mad night time number photos,sampled week nights ratio,entropy on number of photos taken in night time,ar of num of photos in night time lag-1,...,mean number of photos in burst at weekend,mean number of bursts in weekend,median work days number photos,mad work days number photos,sampled weekdays ratio,entropy on number of photos taken in weekdays,ar of num of photos in weekdays lag-1,mean number of photos in burst at weekdays,mean number of bursts in weekdays,User_Register
description,,,,,,,,,,,,,,,,,,,,,
005b6c49-c4ed-438b-9092-ad02ed4d672e,0.0,0.000000,0.251656,1.300914,0.845481,0.0,0.000000,0.233444,5.940174,20.684624,...,1.302730,0.361233,0.0,0.000000,0.220199,5.849773,6.420585,1.210223,0.463557,0.0
007a0785-1321-4112-be89-e6549a4725a2,0.0,0.000000,0.153846,2.791781,1.641026,0.0,0.000000,0.118321,3.696227,34.163954,...,2.676349,0.619608,0.0,0.000000,0.135531,2.751838,56.616557,2.173167,1.238095,1.0
00e37960-250f-4185-94ac-143b319730e3,13.0,10.378216,1.000000,1.209393,2.206897,5.0,4.447807,0.964286,4.226331,-1.000000,...,1.104895,0.344828,10.5,6.671710,0.964286,4.200686,-1.000000,1.170483,1.535714,0.0
01ae3991-50dc-4a67-9c37-cc88ddea90a2,3.0,4.447807,0.577778,1.308057,1.113636,0.0,0.000000,0.422222,3.801977,5.136123,...,1.166667,0.161290,2.0,2.965204,0.533333,3.957295,3.677322,1.238372,0.767442,0.0
01c59096-22f1-40fa-b771-49725b91cc93,7.0,5.930409,0.976190,1.583691,1.725000,3.0,2.965204,0.857143,4.669407,3.885961,...,1.526316,0.400000,5.0,4.447807,0.926829,4.770613,4.693956,1.453608,1.263158,0.0


In [10]:
x = x.set_index('description')

In [12]:
len(x)

1114

In [13]:
len(x[x.User_Register == 1])

379

In [ ]:
def csv_to_df(path):
    x = pd.read_csv(path)
    x.rename(columns={'Unnamed: 0': 'description'}, inplace=True)
    x = x.set_index('description')
    return x
